Домашная работа. Выделение ключевых слов

In [ ]:
!pip install pymorphy2
!pip install summa
!pip install python-rake
!pip install yargy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
import nltk
import numpy as np
import RAKE
import pandas as pd
import json

In [ ]:
from string import digits
from yargy import Parser
from yargy.pipelines import morph_pipeline
from pymorphy2.tokenizers import simple_word_tokenize
from yargy import rule, or_
from yargy.interpretation import fact
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
from yargy import Parser, rule, and_
from yargy.predicates import gram, dictionary

nltk.download('stopwords')
stopword = stopwords.words('russian')
stopword.append('это')
stopword.append('который')
regexp_tokenizer = RegexpTokenizer(r'\w+')
pymorph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv('tags.tsv',sep='\t')
df.head()

,title,abstract,auto_keywords,hand_keywords
0,Глава транспортного департамента: в ближайшие ...,Глава Таллиннского транспортного департамента ...,Таллинн,троллейбус;электробус;общественный транспорт;б...
1,Финансовое неравенство среди жителей Эстонии р...,Рост доходов населения и уровень жизни увеличи...,Департамент статистики Эстонии,финансовое неравенство;бедность;экономическая ...
2,С этого года Касса по безработице будет предла...,С 1 января 2019 года при Кассе по безработице ...,Касса по безработице;SA Innove;Меэлис Паавел;R...,касса по безраюотице;рфнок труда;выбор професс...
3,Под завалами дома в Магнитогорске нашли живого...,Под завалами обрушившегося многоквартирного до...,Москва,Магнитогорст;обрушение дома;вызрыв газа
4,В связи с открытием мемориала в Лахингувялья и...,В связи с открытием памятника битве под Кехра ...,Таллинн;Нарва;Кехра;Elron;Wabadus,изменение расписания;Таллинн


Корпус был взят из подготовленного корпуса для исследования по [ссылке](https://www.clarin.si/repository/xmlui/handle/11356/1403#).
Вспоследствии также самостоятельно размечен. Так как в данном случае в качестве эталонной разметки неразумно выбирать пересечение ручной и предложенной авторами статьи, в качестве эталонной будем использовать ручной вариант. 
Часто в качестве ключевых слов выбирается не событие, а упомянутые локации или деятели

In [ ]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [ ]:
def get_rake_tags(normalized_text, n):
    tags = []
    rake = RAKE.Rake(stopword)
    tag_list = rake.run(normalized_text, maxWords=3, minFrequency=2)

    for tag in tag_list:
      if tag[1] > 0:
        tags.append(tag[0])
    
    return tags[:n]

In [ ]:
def get_textrank_tags(normalized_text, n):
  tag_list = keywords.keywords(normalized_text, language='russian', 
                           additional_stopwords=stopword, scores=True)
  return [i[0] for i in tag_list][:n]

In [ ]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
  sorted_items = sorted_items[:topn]
  score_vals = []
  feature_vals = []

  for idx, score in sorted_items:
      score_vals.append(round(score, 3))
      feature_vals.append(feature_names[idx])

  results= {}

  for idx in range(len(feature_vals)):
      results[feature_vals[idx]]=score_vals[idx]

  return results

In [ ]:
def get_tfidf_tags(vectorizer, feature_names, normalized_text, n):
  tf_idf_vector = vectorizer.transform([normalized_text])
  sorted_items=sort_coo(tf_idf_vector.tocoo())
  tags=extract_topn_from_vector(feature_names,sorted_items, n)
  
  return list(tags.keys())

In [ ]:
def sort_coo(coo_matrix):
  tuples = zip(coo_matrix.col, coo_matrix.data)
  return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [ ]:
def get_tags_docs(frame, tagging_method, n, column='preprocessed'):
  tags = []

  if tagging_method == get_tfidf_tags:
    vectorizer = TfidfVectorizer(stop_words=stopword, smooth_idf=True, 
                             use_idf=True, ngram_range=(1,3))
    vectorizer.fit_transform(frame['abstract'])
    feature_names = vectorizer.get_feature_names()

  for doc in frame[column]:
    text = doc
    if tagging_method != get_tfidf_tags:
      tag_set = tagging_method(text, n)
    else:
      tag_set = tagging_method(vectorizer, feature_names, text, n)
    
    tags.append(tag_set)
  
  return tags

In [ ]:
def get_standart(frame, column='hand_keywords'):
  tags = []
  
  for i in range(0, len(frame)):
    tags_set = [m.parse(w)[0].normal_form for w in str(frame[column][i]).split(';')]
    tags.append(tags_set)
 
  return tags

In [ ]:
def get_metrics(predicted_tags, gold_tags):
  preсisions = []
  recalls = []
  f_scores = []

  for id, tag_set in enumerate(gold_tags):
    doc = gold_tags[id]
    TP = 0
    
    for i in range(0, len(predicted_tags[id])):
      if predicted_tags[id][i] in doc:
        TP+=1
    
    try:
      precision = TP/len(predicted_tags[id])
    except ZeroDivisionError:
      precision = 0
    preсisions.append(precision)
    try:
      recall = TP/len(doc)
    except ZeroDivisionError:
      recall = 0
    recalls.append(recall)
    if precision != 0 and recall != 0:
      f_score = 2*precision*recall/(precision+recall)
    else:
      f_score = 0
    f_scores.append(f_score)
  
  return preсisions, recalls, f_scores

In [ ]:
df['preprocessed'] = df.apply(lambda row: normalize_text(row['abstract']), axis=1)

In [ ]:
standart = get_standart(df)
standart

[['троллейбус',
  'электробус',
  'общественный транспорт',
  'будущее транспорт',
  ''],
 ['финансовое неравенство', 'бедность', 'экономическая статистика'],
 ['касса по безраюотица', 'рфнок труд', 'выбор профессия', 'карьера '],
 ['магнитогорст', 'обрушение дом', 'вызрыв газ'],
 ['изменение расписание', 'таллинн'],
 ['шторм', 'прогноз погода'],
 ['учебный кредить', 'отсрочка', 'законопроект'],
 ['инцидент', 'авария', 'мост'],
 ['планы уехать', 'жизнь заграница', 'миграция'],
 ['алкоголь', 'статистика'],
 ['мобильное приложение', 'статистика', 'зависимсть', 'цифровая грамотность'],
 ['чп'],
 ['отравление', 'пищевой департмент'],
 ['фейерверк', 'пиротехник'],
 ['кнд', 'независимость тайвань', 'суверенитет', 'применение военной сила'],
 ['nan'],
 ['дтп'],
 ['дтп'],
 ['нарва', 'переезд', 'эстонский язык'],
 ['нарушение мобильной связь', 'шторм'],
 ['пневматический пистолет', 'убийство'],
 ['отмена рейс', 'аэропорт'],
 ['ребёнок', 'роды'],
 ['радар фиксации скорость'],
 ['освободительная 

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stopword, smooth_idf=True, 
                             use_idf=True, ngram_range=(1,2), min_df=0.2,
                             max_df=0.8)
vectorizer.fit_transform(df['preprocessed'])
feature_names = vectorizer.get_feature_names()

In [ ]:
rake_tags = get_tags_docs(df, get_rake_tags, 15)
textrank_tags = get_tags_docs(df, get_textrank_tags, 15)
tfidf_tags = get_tags_docs(df, get_tfidf_tags, 15)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
rake_tags

[['сторона'],
 [],
 ['карьера весь желать',
  'sa innove',
  'безработица',
  'карьера',
  'консультирование',
  'число',
  'школьник'],
 [],
 ['действовать временной расписание', '6 январь', 'мероприятие'],
 ['5 м /'],
 ['процентный ставка',
  'учебный кредит',
  'брать кредит',
  'кредит',
  'год',
  'выгодный',
  'студент',
  'находиться',
  'поручитель',
  'требование',
  'наличие'],
 ['причина авария уточняться', 'зеландия', 'следовать'],
 ['уехать', 'человек', 'эстония', 'жить', 'австралия', 'недавно', 'какой-'],
 ['алкоголь'],
 ['приложение напомнить', 'использовать смартфон', 'время', 'неделя'],
 [],
 ['середина декабрь', 'курс произойти', 'слово'],
 [],
 ['независимость тайвань', 'китай', 'тупик', 'китаец'],
 ['редакция оставлять', 'праздник сад', 'конкурс', 'сад', 'украсить', 'право'],
 [],
 [],
 ['— сказать кяосаар', 'нарва', 'начинать'],
 ['данный момент'],
 ['кохтла-яров', 'задержать', 'мужчина'],
 ['хольтс сказать', 'также', 'рига'],
 ['автоматически снять штаны',
  'опус

In [ ]:
textrank_tags

[['эксплуатация троллейбус',
  'харьо',
  'электроавтобус',
  'случай',
  'десять',
  'большой',
  'троллейбусный',
  'иметь',
  'аккумулятор',
  'особый',
  'таллиннский',
  'скорее',
  'служить',
  'касаться'],
 ['рост доход',
  'показатель',
  'жизнь',
  'снизиться',
  'региональный',
  'ымблус',
  'положение',
  'писать',
  'скорее'],
 ['услуга',
  'безработица',
  'год',
  'касса',
  'молодёжь',
  'центр карьера',
  'человек',
  'стать',
  'изменение',
  'качество',
  'методика',
  'заниматься',
  'эстонский'],
 ['обнаружить',
  'спасатель',
  'взрыв',
  'дом',
  'магнитогорск',
  'наслать',
  'городской'],
 ['открытие памятник битва',
  'случай',
  'данный время',
  'круглосуточный инфотелефон',
  'wabadus',
  'эстонский',
  'временной'],
 ['имя аапеть',
  'начаться ночью шторм',
  'aapeli',
  'ветер',
  'м',
  'синоптик',
  'служба',
  'прежний',
  'остров'],
 ['кредит',
  'год',
  'законопроект',
  'учебный',
  'закон',
  'финансовый',
  'условие',
  'кредитный',
  'мочь',
  'у

In [ ]:
tfidf_tags

[['эксплуатация',
  'харьо',
  'случай',
  'сторона',
  'сказать',
  'новый',
  'иметь',
  'друг',
  'десять',
  'год',
  'электромотор',
  'эксплуатация намного',
  'чиновник',
  'харьо delfi чиновник',
  'харьо delfi'],
 ['рост',
  'показатель',
  'ымблус',
  'слово',
  'прошлый год',
  'прошлый',
  'положение',
  'находиться',
  'население',
  'мюйрсоо',
  'многое',
  'весь',
  'taga',
  'arvude taga',
  'arvude'],
 ['касса',
  'консультирование',
  'информирование консультирование',
  'информирование',
  'год',
  'предлагать',
  'изменение',
  'весь',
  'sa innove',
  'sa',
  'rajaleidja sa innove',
  'rajaleidja sa',
  'rajaleidja',
  'innove',
  'эстонский'],
 ['дом',
  'человек судьба 27',
  'человек судьба',
  'судьба 27',
  'спустя сутки',
  'подъезд',
  'обрушение',
  'неизвестный',
  'лечение',
  'доставить',
  'действительно сегодня',
  'данные',
  'весь',
  '120 человек судьба',
  '120 человек'],
 ['таллинн',
  'поезд',
  'расписание',
  'действовать',
  'автобус',
  'учас

In [ ]:
def get_mean_scores(tags, standart):
  tags_scores = get_metrics(tags, standart)
  tags_precision = np.mean(tags_scores[0])
  tags_recall = np.mean(tags_scores[1])
  tags_f_scores = np.mean(tags_scores[2])
  
  return 'precision: {}, recall: {}, f-score: {}'.format(
      tags_precision, tags_recall, tags_f_scores)

In [ ]:
get_mean_scores(rake_tags, standart)

'precision: 0.0725907029478458, recall: 0.09732142857142856, f-score: 0.07222813367771351'

In [ ]:
get_mean_scores(textrank_tags, standart)

'precision: 0.03544283097854526, recall: 0.17083333333333334, f-score: 0.05524249173353552'

In [ ]:
get_mean_scores(tfidf_tags, standart)

'precision: 0.04196428571428572, recall: 0.30178571428571427, f-score: 0.07244677729391351'

Можно заметить, что в эталонной разметке часто встречаются некоторые речевые паттерны, в частности, частиречевые. Чаще всего это существительное, существительное + прилагательное, существительное + существительное 

In [ ]:
Name = fact(
    'Name',
    ['first', 'last']
)


NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)


AdjN = rule(gram('ADJF'), gram('NOUN'))
NN = rule(gram('NOUN'), gram('NOUN'))
N = rule(gram('NOUN'))

grammar = Parser(
    or_(
        NAME,
        AdjN,
        NN,
        N
    )
)

In [ ]:
def extract_tags(tag_set):
  new_tags = []
  
  for tag in tag_set:
    if not tag.isascii():
      new_tags.append(tag)
      for match in grammar.findall(tag):
        new_tag = ' '.join([_.value for _ in match.tokens])
        new_tags.append(new_tag)
  
  return new_tags

In [ ]:
def get_new_tag_sets(tags):
  new_tag_sets = []

  for tag in tags:
      new_tag_sets.append(extract_tags(tag))
  
  return new_tag_sets

In [ ]:
new_rake = get_new_tag_sets(rake_tags)
new_textrank = get_new_tag_sets(textrank_tags)
new_tfidf = get_new_tag_sets(tfidf_tags)

In [ ]:
get_mean_scores(new_rake, standart)

'precision: 0.08253466771323914, recall: 0.2095238095238095, f-score: 0.10630662646791679'

In [ ]:
get_mean_scores(new_textrank, standart)

'precision: 0.040932858670272854, recall: 0.3148809523809524, f-score: 0.06997653779911844'

In [ ]:
get_mean_scores(new_tfidf, standart)

'precision: 0.04840135947183805, recall: 0.5708333333333334, f-score: 0.08834413685816622'